## Missionaries and Carnivals Problem (in BFS, DLS, DFS, IDS, ILS, UCS)

In [1]:
import sys
import time
import json
import heapq
import random
from collections import deque

In [2]:
def is_valid(state):
  farmer, wolf, goat, cabbage, boat = state
  if goat == wolf and farmer != goat:
    return False
  if goat == cabbage and farmer != goat:
    return False
  return True

In [3]:
def get_successors(state):
  farmer, wolf, goat, cabbage, boat = state
  moves = []
  new_boat = 'R' if boat == 'L' else 'L'
  new_state = (new_boat if farmer == boat else farmer, wolf, goat, cabbage, new_boat)
  if is_valid(new_state):
    moves.append((new_state, "Farmer crosses alone"))
    # Farmer takes wolf
  if farmer == wolf == boat:
    new_state = (new_boat, new_boat, goat, cabbage, new_boat)
  if is_valid(new_state):
    moves.append((new_state, "Farmer takes Wolf"))
  # Farmer takes goat
  if farmer == goat == boat:
    new_state = (new_boat, wolf, new_boat, cabbage, new_boat)
  if is_valid(new_state):
    moves.append((new_state, "Farmer takes Goat"))
  # Farmer takes cabbage
  if farmer == cabbage == boat:
    new_state = (new_boat, wolf, goat, new_boat, new_boat)
  if is_valid(new_state):
    moves.append((new_state, "Farmer takes Cabbage"))
  return moves

In [6]:
def state_to_str(state):
  # Converting state tuple to string
  names = ['Farmer', 'Wolf', 'Goat', 'Cabbage', 'Boat']
  return ', '.join(f"{name}:{pos}" for name, pos in zip(names, state))

In [7]:
def read_input(filename):
  with open(filename, 'r') as f:
    params = json.load(f)
  return params

def write_log(logfile, text):
  # Appending the log file
  with open(logfile, 'a') as f:
    f.write(text + '\n')

def print_path(path, actions, logfile):
  write_log(logfile, f"Solution found in {len(path)-1} steps:")
  for i, (state, action) in enumerate(zip(path[1:], actions)):
    write_log(logfile, f"Step {i+1}: {action} -> {state_to_str(state)}")


### BFS (Breadth First Search)

In [8]:
def bfs(start, goal, logfile):
# Exploring the shallow nodes first
  queue = deque()
  queue.append((start, [start], []))
  visited = set()
  step = 0
  while queue:
    state, path, actions = queue.popleft()
    write_log(logfile, f"BFS Step {step}: {state_to_str(state)}")
    if state == goal:
      return path, actions
    visited.add(state)
    for succ, action in get_successors(state):
        if succ not in visited:
          queue.append((succ, path + [succ], actions + [action]))
    step += 1
  return None, None

### DFS (Depth First Search)

Exploring as far as possible along each branch before backtracking using a stack (LIFO). If depth_limit is set, the program performs a Depth-Limited Search (DLS).
    

In [ ]:
def dfs(start, goal, logfile, depth_limit=None):
  stack = [(start, [start], [], 0)]
  visited = set()
  step = 0
  while stack:
    state, path, actions, depth = stack.pop()
    write_log(logfile, f"DFS Step {step}: {state_to_str(state)} (depth {depth})")
    if state == goal:
      return path, actions
    if depth_limit is not None and depth >= depth_limit:
      continue
    visited.add(state)
    for succ, action in get_successors(state):
      if succ not in visited:
        stack.append((succ, path + [succ], actions + [action], depth + 1))
    step += 1
  return None, None

### DLS (Depth Limited Search)

In [ ]:
def dls(start, goal, logfile, limit):
  # calling DFS with a depth-limit
    return dfs(start, goal, logfile, depth_limit=limit)

### IDS (Iterative Deepening Search)

Repeatedly applying DLS with increasing depth limits.
    

In [ ]:
def ids(start, goal, logfile, max_depth):
  for depth in range(max_depth + 1):
    write_log(logfile, f"IDS: Trying depth {depth}")
    path, actions = dfs(start, goal, logfile, depth_limit=depth)
    if path:
      return path, actions
  return None, None

### UCS (Uniform Cost Search)

Expanding the least-codenode using a priority queue.

In [ ]:
def ucs(start, goal, logfile, cost_fn):
  heap = []
  heapq.heappush(heap, (0, start, [start], []))
  visited = set()
  step = 0
  while heap:
    cost, state, path, actions = heapq.heappop(heap)
    write_log(logfile, f"UCS Step {step}: {state_to_str(state)} (cost {cost})")
    if state == goal:
      return path, actions
    visited.add(state)
    for succ, action in get_successors(state):
      if succ not in visited:
        heapq.heappush(heap, (cost + cost_fn(state, succ), succ, path + [succ], actions + [action]))
    step += 1
  return None, None

### ILS (Iterative Local Search)

Performing multiple DFS runs with randomization and restarting.

In [ ]:
def ils(start, goal, logfile, max_restarts):
  best_path = None
  best_actions = None
  for restart in range(max_restarts):
    write_log(logfile, f"ILS: Restart {restart}")
    stack = [(start, [start], [], 0)]
    visited = set()
    while stack:
      state, path, actions, depth = stack.pop()
      if state == goal:
        if not best_path or len(path) < len(best_path):
          best_path = path
          best_actions = actions
        break
      visited.add(state)
      successors = get_successors(state)
      random.shuffle(successors)
      for succ, action in successors:
        if succ not in visited:
          stack.append((succ, path + [succ], actions + [action], depth + 1))
  return best_path, best_actions

## Main Function

In [ ]:
def main():
  # Setting parameters directly or reading from a single input file
  params = read_input("input.json")
  start = tuple(params["start"])
  goal = tuple(params["goal"])
  logfile = "all_algorithms_output.txt"
  # Clearing the output file at the start
  open(logfile, 'w').close()

# BFS
  write_log(logfile, "\n========== BFS ==========")
  t0 = time.time()
  path, actions = bfs(start, goal, logfile)
  t1 = time.time()
  if path:
    print_path(path, actions, logfile)
  else:
    write_log(logfile, "No solution found.")
  write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")

# DFS
  write_log(logfile, "\n========== DFS ==========")
  t0 = time.time()
  path, actions = dfs(start, goal, logfile)
  t1 = time.time()
  if path:
    print_path(path, actions, logfile)
  else:
    write_log(logfile, "No solution found.")
  write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")

# DLS with different limits
  for limit in [5, 10, 20]:
    write_log(logfile, f"\n========== DLS (limit={limit}) ==========")
    t0 = time.time()
    path, actions = dls(start, goal, logfile, limit)
    t1 = time.time()
    if path:
      print_path(path, actions, logfile)
    else:
      write_log(logfile, "No solution found.")
    write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")

# IDS with different max_depth
  for max_depth in [5, 10, 20]:
    write_log(logfile, f"\n========== IDS (max_depth={max_depth}) ==========")
    t0 = time.time()
    path, actions = ids(start, goal, logfile, max_depth)
    t1 = time.time()
    if path:
      print_path(path, actions, logfile)
    else:
      write_log(logfile, "No solution found.")
  write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")

    # UCS
    write_log(logfile, "\n========== UCS ==========")
    t0 = time.time()
    def cost_fn(s1, s2): return params.get("step_cost", 1)
    path, actions = ucs(start, goal, logfile, cost_fn)
    t1 = time.time()
    if path:
        print_path(path, actions, logfile)
    else:
        write_log(logfile, "No solution found.")
    write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")

    # ILS with different restarts
    for restarts in [3, 5, 10]:
        write_log(logfile, f"\n========== ILS (restarts={restarts}) ==========")
        t0 = time.time()
        path, actions = ils(start, goal, logfile, restarts)
        t1 = time.time()
        if path:
            print_path(path, actions, logfile)
        else:
            write_log(logfile, "No solution found.")
        write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")